In [66]:
# library imports 
import sys
from importlib import reload
import findspark
import customHelpers as helper
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType,IntegerType, FloatType,BooleanType,DateType

reload(helper)

<module 'customHelpers' from '/Users/nagibshah/dev/COMP5349_AmazonProductReviewAnalysis/notebooks/customHelpers.py'>

In [2]:
# initialise the session 

spark = SparkSession \
    .builder \
    .appName("Amazon Product Review Analysis") \
    .getOrCreate()

#sc = SparkContext(appName="Amazon Product Review Analysis")

## Load Dataset 

| Column | Description | 
| :--- | :--- |
| marketplace | 2 letter country code of the marketplace where the review was written. |
| customer_id | Random identifier that can be used to aggregate reviews written by a single author. |
| review_id | The unique ID of the review. |
| product_id | The unique Product ID the review pertains to. In the multilingual dataset the reviews for the same product in different countries can be grouped by the same product_id. | 
| product_parent | Random identifier that can be used to aggregate reviews for the same product. |
| product_title | Title of the product. | 
| product_category | Broad product category that can be used to group reviews (also used to group the dataset into  coherent parts). | 
| star_rating | the 1-5 star rating of the review. | 
| helpful_votes | Number of helpful votes. | 
| total_votes | Number of total votes the review received. | 
| vine | Review was written as part of the Vine program. |
| verified_purchase | The review is on a verified purchase. |
| review_headline | The title of the review. |
| review_body | The review text. |
| review_date | The date the review was written | 


DATA FORMAT
Tab ('\t') separated text file, without quote or escape characters.
First line in each file is header; 1 line corresponds to 1 record.


In [3]:
# load the data set 
#review_data = '../data/sample_us.tsv'
# actual data load - PERFORMANCE WARNING ON LOCAL MACHINE
review_data = '../data/amazon_reviews_us_Music_v1_00.tsv'

aws_product_review_schema = StructType([
    StructField("marketplace", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("review_id", StringType(), True),
    StructField("product_id",StringType(),True),
    StructField("product_parent",StringType(),False),
    StructField("product_title", StringType(), False),
    StructField("product_category", StringType(), False),
    StructField("star_rating", IntegerType(), False),
    StructField("helpful_votes",IntegerType(),False),
    StructField("total_votes", IntegerType(), False),
    StructField("vine",StringType(),False),
    StructField("verified_purchase", StringType(), False),
    StructField("review_headline", StringType(), False),
    StructField("review_body", StringType(), False),
    StructField("review_date",DateType(),False)])

aws_product_review_schema_limited = StructType([
    StructField("customer_id", StringType(), True),
    StructField("review_id", StringType(), True),
    StructField("product_id",StringType(),True),
    StructField("product_title", StringType(), False),
    StructField("product_category", StringType(), False),
    StructField("star_rating", IntegerType(), False),
    StructField("helpful_votes",IntegerType(),False),
    StructField("total_votes", IntegerType(), False),
    StructField("review_headline", StringType(), False),
    StructField("review_body", StringType(), False),
    StructField("review_date",DateType(),False)])

%time awsProductReview_raw_data = spark.read.csv(review_data,header=True,sep="\t",schema=aws_product_review_schema)



CPU times: user 2.41 ms, sys: 1.29 ms, total: 3.69 ms
Wall time: 1.22 s


In [4]:
# when testing in local machine only 
print(awsProductReview_raw_data.count())
# limit to 1 mil
awsProductReview_raw_data = awsProductReview_raw_data.limit(1000000)
print(awsProductReview_raw_data.count())

4751577
1000000


In [5]:
awsProductReview_raw_data.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [6]:
dfProductReview = awsProductReview_raw_data.drop('vine').drop('verified_purchase') \
                    .drop('product_parent').drop('marketplace')
dfProductReview.show(5)


+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+--------------------+--------------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|     review_headline|         review_body|review_date|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+--------------------+--------------------+-----------+
|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|Whatever's for Us...|           Music|          5|            0|          0|          Five Stars|Love this CD alon...| 2015-08-31|
|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|Same Trailer Diff...|           Music|          5|            0|          0|A new fave in our...|This is the album...| 2015-08-31|
|   45946560| R9PYL3OYH55QY|B001GCZXW6| Soaring (Jazz Club)|           Music|          5|            0|          1|          Five Stars|  Excellent / 

In [7]:
# remove rows with no review text 
print("number of rows before filter: {0}".format(dfProductReview.count()))
dfFilteredReviews = dfProductReview.na.drop(subset=["review_body"])
print("number of rows after filter: {0}".format(dfFilteredReviews.count()))

number of rows before filter: 1000000
number of rows after filter: 999701


## Stage One: Overall statistics

### Produce overall summary statistics of the data set, in particular,

* the total number of reviews
* the number of unique users
* the number of unique products

In [8]:
from pyspark.sql.functions import col, count, countDistinct

dfOverallStats = dfProductReview.agg(countDistinct("customer_id").alias("unique_customers"), \
                    countDistinct("product_id").alias("unique_products"), \
                    count(col="review_id").alias("total_reviews")) \

dfOverallStats.show()

+----------------+---------------+-------------+
|unique_customers|unique_products|total_reviews|
+----------------+---------------+-------------+
|          461198|         297940|      1000000|
+----------------+---------------+-------------+



### For user-review distribution, you are asked to find out:

* the largest number of reviews published by a single user
* the top 10 users ranked by the number of reviews they publish
* the median number of reviews published by a user

In [9]:
from pyspark.sql.window import Window
from pyspark.sql.functions import count
import pyspark.sql.functions as F

dfUserReviewCounts = helper.distributionStats(dfRecords=dfProductReview.select("customer_id", "review_id"), \
                                              partitionBy="customer_id",countBy="review_id", \
                                              returnCountName="total_reviews")
print("Top Reviewer:")
dfUserReviewCounts.show(1)
print("Top 10 Reviewers:")
dfUserReviewCounts.show(10)

Top Reviewer:
+-----------+-------------+
|customer_id|total_reviews|
+-----------+-------------+
|   38214553|         1497|
+-----------+-------------+
only showing top 1 row

Top 10 Reviewers:
+-----------+-------------+
|customer_id|total_reviews|
+-----------+-------------+
|   38214553|         1497|
|   15536614|         1224|
|   18116317|         1053|
|    7080939|          921|
|   14539589|          723|
|   42836721|          652|
|   47924228|          604|
|   15725862|          594|
|   47423754|          558|
|   50736950|          513|
+-----------+-------------+
only showing top 10 rows



In [10]:
# median reviews 
# no median finder in spark... do we need to implement using RDD?

user_review_median=dfUserReviewCounts.approxQuantile("total_reviews", [0.50], 0)[0]
print("median number of {0} reviews published by user".format(user_review_median))

median number of 1.0 reviews published by user


### For product-review distribution, you are asked to find out:
    
* the largest number of reviews written for a single product
* the top 10 products ranked by the number of reviews they have
* the median number of reviews a product has

In [11]:
dfProductReviewCounts = helper.distributionStats(dfRecords=dfProductReview.select("product_id", "review_id"), \
                                              partitionBy="product_id",countBy="review_id", \
                                              returnCountName="total_reviews")
print("Top Product By Review:")
dfProductReviewCounts.show(1)
print("Top 10 Products by Reviews:")
dfProductReviewCounts.show(10)

Top Product By Review:
+----------+-------------+
|product_id|total_reviews|
+----------+-------------+
|B00MIA0KGY|         2699|
+----------+-------------+
only showing top 1 row

Top 10 Products by Reviews:
+----------+-------------+
|product_id|total_reviews|
+----------+-------------+
|B00MIA0KGY|         2699|
|B00NEJ7MMI|         2420|
|B00MRHANNI|         1513|
|B00H3GZMIE|         1277|
|B00MU79IL8|         1172|
|B00UCFVIDQ|         1114|
|B00KLF5J64|         1038|
|B00EDY5KTA|         1008|
|B00NQKWAIQ|          997|
|B00007KWHG|          826|
+----------+-------------+
only showing top 10 rows



In [35]:
# median reviews 
# no median finder in spark... do we need to implement using RDD?

product_review_median=int(dfProductReviewCounts.approxQuantile("total_reviews", [0.5], 0)[0])
print("median number of {0} reviews per product".format(product_review_median))

median number of 1 reviews per product


## Stage Two: Filtering Unwanted Data

filter reviews based on length, reviewer and product feature. In particular, the following reviews should be removed:

* reviews with less than two sentences in the review body.
* reviews published by users with less than median number of reviews published
* reviews from products with less than median number of reviews received

NOTE: Sentence Segmentation Using: NLTK

In [13]:
# reviews with less than 2 sentences in review_body
# convert to RDD and carry out a filter to remove rows with less than 2 sentences 

print("number of rows before filter: {0}".format(dfFilteredReviews.count()))

dfFilteredReviews = dfFilteredReviews.filter(helper.FilterSentences('review_body'))

#reviewRdd = dfProductReview.rdd.map(list).filter(helper.FilterSentences)
#print(reviewRdd.take(1))
# convert back to DF 
#dfFilteredReviews = spark.createDataFrame(reviewRdd, aws_product_review_schema_limited)
#dfFilteredReviews = reviewRdd.toDF(schema=aws_product_review_schema_limited)
dfFilteredReviews.show(1)
dfFilteredReviews.cache()

print("number of rows post filter: {0}".format(dfFilteredReviews.count()))

number of rows before filter: 999701
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+--------------------+--------------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|     review_headline|         review_body|review_date|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+--------------------+--------------------+-----------+
|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|Same Trailer Diff...|           Music|          5|            0|          0|A new fave in our...|This is the album...| 2015-08-31|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+--------------------+--------------------+-----------+
only showing top 1 row

number of rows post filter: 329468


In [14]:
# user review filter 
print("number of rows before filter: {0}".format(dfFilteredReviews.count()))

window = Window.partitionBy("customer_id")
dfFilteredReviews = dfFilteredReviews \
    .withColumn("review_count", count("review_id") \
    .over(window)) \
    .filter(col("review_count") >= user_review_median) \
    .drop("review_count")

print("number of rows post filter: {0}".format(dfFilteredReviews.count()))

number of rows before filter: 329468
number of rows post filter: 329468


In [15]:
# product review filter
print("number of rows before filter: {0}".format(dfFilteredReviews.count()))

window = Window.partitionBy("product_id")
dfFilteredReviews = dfFilteredReviews \
    .withColumn("review_count", count("review_id") \
    .over(window)) \
    .filter(col("review_count") >= product_review_median) \
    .drop("review_count")

print("number of rows post filter: {0}".format(dfFilteredReviews.count()))

number of rows before filter: 329468
number of rows post filter: 329468


In [16]:
dfFilteredReviews.show(5)

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+--------------------+--------------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|     review_headline|         review_body|review_date|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+--------------------+--------------------+-----------+
|   16615744|R36G0ETK84RMNN|5552936752|Scherrer/Fritz: S...|           Music|          4|            1|          1|Swiss Orchestral ...|Little is known a...| 2015-08-08|
|   40083442| R6A3LEKK38HG6|B0000004IT|        Singles 1-12|           Music|          5|            0|          0| Melvins Starter Kit|Some might say, i...| 2014-07-03|
|   20890474|R3RQWT6V0NHDSY|B0000005PU|        Mi Vida Loca|           Music|          4|            2|          2|    Delightful Music|Gaffney was a 

In [17]:
print("Original Number of Rows before cleanup: {0}".format(dfProductReview.count()))
print("Number of rows after all filters applied: {0}".format(dfFilteredReviews.count()))

Original Number of Rows before cleanup: 1000000
Number of rows after all filters applied: 329468


In [18]:
dfFilteredReviews.cache()

DataFrame[customer_id: string, review_id: string, product_id: string, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, review_headline: string, review_body: string, review_date: date]

#### After filtering out the above, find out:

* top 10 users ranked by median number of sentences in the reviews they have published
* top 10 products ranked by median number of sentences in the reviews they have received

In [67]:
# top 10 users ranked by median number of sentences in the reviews they have published
#dfTop10UsersBySents = helper.getTopBySentNumber(dfRecords=dfFilteredReviews, topnCol="customer_id", \
#                                                textCol="review_body",n=10)

dfTop10UsersBySents = helper.getTopBySentMedian(dfRecords=dfFilteredReviews,partitionBy="customer_id", \
                                                textCol="review_body",medianColName="median_sents",n=10)
#dfTop10UsersBySents.select("customer_id","product_id","product_title","review_body","median_sents").show(10)
dfTop10UsersBySents.show()


+-----------+------------+
|customer_id|median_sents|
+-----------+------------+
|   41838529|         984|
|   51970720|         454|
|   51865782|         440|
|   52672392|         311|
|   50595705|         211|
|   17821650|         183|
|   36934717|         171|
|   29705444|         163|
|   13551370|         157|
|   14678937|         155|
+-----------+------------+



In [69]:
# top 10 products ranked by median number of sentences in the reviews they have received
#dfTop10ProductsBySents = helper.getTopBySentNumber(dfRecords=dfFilteredReviews, topnCol="product_id", \
#                                                textCol="review_body",n=10)
dfTop10ProductsBySents = helper.getTopBySentMedian(dfRecords=dfFilteredReviews,partitionBy="product_id", \
                                                textCol="review_body",medianColName="median_sents",n=10)

dfTop10ProductsBySents.show(10)

+----------+------------+
|product_id|median_sents|
+----------+------------+
|B00LTQ5EVY|         984|
|B00000411D|         302|
|B000001G8Z|         295|
|B002QZPVBK|         289|
|B00HSP0P0U|         274|
|B005MJVMO2|         274|
|B005OZBTWE|         273|
|B000005986|         271|
|B000004119|         269|
|B000003425|         266|
+----------+------------+



In [104]:
# save the cleaned and filtered dataframe to file system 

#dfFilteredReviews.coalesce(1).write.format("parquet") \
#    .option("header", "true").saveAsTable('filteredReviews',mode="overwrite")
dfFilteredReviews.coalesce(1).write.csv("../output",mode="overwrite",header=True,sep="\t")

## Stage 3 Similarity analysis with Sentence Embedding

perform similarity analysis on the review sentences. The analysis involves segmenting review body into multiple sentences; encoding each sentence as vector so that the distance between pair of sentences can be computed.

### Positive vs. Negative Reviews

* pick a product from the top 10 products in stage 1
* Create a positive and negative class of reviews using the rating 
    - Positive Class - rate >=4 
    - Negative Class - rate <= 2
    - for each review, extracting the review body part and segment it into multiple sentences.
    - encode the sentences using google universal encoder


In [112]:
# load the base/filtered dataset created earlier 
filtered_data = "../output/part-*.csv"
dfBaseDataset = spark.read.csv(filtered_data,header=True,sep="\t",schema=aws_product_review_schema_limited)

In [114]:
# product from top 10 by review number 
base_product_id = "B00MIA0KGY"

dfPositiveClass = dfBaseDataset.where((col("product_id") == base_product_id) & (col("star_rating") >= 4))
dfNegativeClass = dfBaseDataset.where((col("product_id") == base_product_id) & (col("star_rating") <= 2))

In [116]:
print("number of positives: {0}".format(dfBaseDataset.count()))
print("number of positives: {0}".format(dfPositiveClass.count()))
print("number of negatives: {0}".format(dfNegativeClass.count()))

number of positives: 110098
number of positives: 214
number of negatives: 16


In [134]:
# for each review, extract the review body part and segment it into multiple sentences
# extract the positive sentences
dfPosSents = dfPositiveClass.select("review_id","review_body") \
    .withColumn("sentences", helper.GenerateSentences("review_body")) \
    .select("review_id", F.explode_outer("sentences"))


### Extract the sentences - similar to flatMap

In [135]:
# extract the negative sentences
dfNegSents = dfNegativeClass.select("review_id","review_body") \
    .withColumn("sentences", helper.GenerateSentences("review_body")) \
    .select("review_id", F.explode_outer("sentences"))

In [137]:
print("Positive sententences extracted: {0}".format(dfPosSents.count()))
print("Negative sententences extracted: {0}".format(dfNegSents.count()))

dfPosSents.show(5)
dfNegSents.show(5)

Positive sententences extracted: 1324
Negative sententences extracted: 139
+--------------+--------------------+
|     review_id|                 col|
+--------------+--------------------+
|R338L3ESXHT0XJ|                Wow!|
|R338L3ESXHT0XJ|The absolutely pe...|
|R338L3ESXHT0XJ|She has never sou...|
|R338L3ESXHT0XJ|I would not be su...|
|R338L3ESXHT0XJ|Impossible to pic...|
+--------------+--------------------+
only showing top 5 rows

+--------------+--------------------+
|     review_id|                 col|
+--------------+--------------------+
|R2D3O3SJ7R34VV|         Dullsville.|
|R2D3O3SJ7R34VV|All the wrong son...|
|R2D3O3SJ7R34VV|Streisand even  h...|
|R2D3O3SJ7R34VV|             BORING.|
|R2D3O3SJ7R34VV|        No artistry.|
+--------------+--------------------+
only showing top 5 rows



### Encoding the sentences - google universal encoder 